In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [ ]:
# General
import numpy as np
import pandas as pd
import os
import scipy.stats as st
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

# Evalaluation
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

# Grid
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.feature_selection import RFE
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [ ]:
dep = 'SalePrice'

# Train
train_df = pd.read_csv("../input/feature-engineering-and-pre-processing-house-data/house_train.csv", index_col='Id')
traindex = train_df.index
# Log conversion
train_df[dep] = np.log(train_df[dep])

# Test
test_df = pd.read_csv("../input/feature-engineering-and-pre-processing-house-data/house_test.csv", index_col='Id')
testdex = test_df.index
test_df[dep] = 0

# Combine
df = pd.concat([train_df,test_df],axis=0)

print("Train Shape:", train_df.shape)
print("Test Shape:",test_df.shape)
print("ALL Shape:", df.shape)
del test_df, train_df

In [ ]:
cat_vars = list(df.loc[:,df.dtypes=="object"].columns)

contin_vars = [x for x in df.columns if x not in cat_vars + [dep]]

n = len(df); n

In [ ]:
for v in cat_vars:
    df[v] = df[v].astype('category').cat.as_ordered()

for v in contin_vars:
    df[v] = df[v].fillna(0).astype('float32')

In [ ]:
cat_sz = [(c, len(df[c].cat.categories)+1) for c in cat_vars]
print(cat_sz)

emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]

In [ ]:
submission_df = pd.DataFrame(index=testdex)

# Indepedent and Dependent
X = df.loc[traindex,:]
testing = df.loc[testdex,:]
del df

In [ ]:
df, y, nas, mapper = proc_df(X, dep, do_scale=True)

df_test, _, nas, mapper = proc_df(testing, dep, do_scale=True,
                                  mapper=mapper, na_dict=nas)

In [ ]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(len(df) * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [ ]:
def inv_y(a): return np.exp(a)

def exp_rmspe(y_pred, targ):
    targ = inv_y(targ)
    pct_var = (targ - inv_y(y_pred))/targ
    return math.sqrt((pct_var**2).mean())

max_log_y = np.max(y)
y_range = (0, max_log_y*1.2)

In [ ]:
PATH= "../input/feature-engineering-and-pre-processing-house-data/"
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test)

In [ ]:
??md.get_learner

In [ ]:
TMP_PATH = "/tmp/tmp"
MODEL_PATH = "/tmp/model/"
PATH_WRITE = '../input/rossmann-store-extra/'

m = md.get_learner(emb_szs, len(df.columns)-len(cat_vars),
                   0.04, 1, [1000,500], [0.001,0.01], y_range=y_range,
                  tmp_name=f"{PATH_WRITE}tmp", models_name=f"{PATH_WRITE}models")
m.summary()

In [ ]:
df.head()